In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn.functional as F

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
transform = transforms.Compose([
    transforms.Pad(4),
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32),
    transforms.ToTensor()])

In [4]:
# CIFAR-10 dataset
train_dataset = torchvision.datasets.CIFAR10(root='./data/',
                                             train=True, 
                                             transform=transform,
                                             download=True)

test_dataset = torchvision.datasets.CIFAR10(root='./data/',
                                            train=False, 
                                            transform=transforms.ToTensor())

# Data loader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=100, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=100, 
                                          shuffle=False)

Extracting ./data/cifar-10-python.tar.gz to ./data/


In [5]:
import torch.nn as nn
class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_planes, planes, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(
            in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3,
                               stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion*planes,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion*planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out


In [6]:
class Bottleneck(nn.Module):
    expansion = 4

    def __init__(self, in_planes, planes, stride=1):
        super(Bottleneck, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3,
                               stride=stride, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)
        self.conv3 = nn.Conv2d(planes, self.expansion *
                               planes, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(self.expansion*planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion*planes,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion*planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = F.relu(self.bn2(self.conv2(out)))
        out = self.bn3(self.conv3(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out


In [7]:
class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=10):
        super(ResNet, self).__init__()
        self.in_planes = 64

        self.conv1 = nn.Conv2d(3, 64, kernel_size=3,
                               stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)
        self.linear = nn.Linear(512*block.expansion, num_classes)

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes * block.expansion
        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = F.avg_pool2d(out, 4)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out

In [8]:
model = ResNet(BasicBlock, [2, 2, 2, 2]).to(device)#here we can give whether implement basicblock or bottleneck

In [9]:
# Loss and optimizer
num_epochs = 80
learning_rate = 0.001
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [10]:
# For updating learning rate
def update_lr(optimizer, lr):    
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

In [11]:
# Train the model
total_step = len(train_loader)
curr_lr = learning_rate
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i+1) % 100 == 0:
            print ("Epoch [{}/{}], Step [{}/{}] Loss: {:.4f}"
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

    # Decay learning rate
    if (epoch+1) % 20 == 0:
        curr_lr /= 3
        update_lr(optimizer, curr_lr)

Epoch [1/80], Step [100/500] Loss: 1.4909
Epoch [1/80], Step [200/500] Loss: 1.7373
Epoch [1/80], Step [300/500] Loss: 1.3315
Epoch [1/80], Step [400/500] Loss: 1.2161
Epoch [1/80], Step [500/500] Loss: 1.2118
Epoch [2/80], Step [100/500] Loss: 0.9498
Epoch [2/80], Step [200/500] Loss: 0.9058
Epoch [2/80], Step [300/500] Loss: 1.0373
Epoch [2/80], Step [400/500] Loss: 0.8527
Epoch [2/80], Step [500/500] Loss: 0.8817
Epoch [3/80], Step [100/500] Loss: 0.8233
Epoch [3/80], Step [200/500] Loss: 0.8272
Epoch [3/80], Step [300/500] Loss: 0.7172
Epoch [3/80], Step [400/500] Loss: 0.7266
Epoch [3/80], Step [500/500] Loss: 0.6662
Epoch [4/80], Step [100/500] Loss: 0.6436
Epoch [4/80], Step [200/500] Loss: 0.5806
Epoch [4/80], Step [300/500] Loss: 0.6699
Epoch [4/80], Step [400/500] Loss: 0.6287
Epoch [4/80], Step [500/500] Loss: 0.7519
Epoch [5/80], Step [100/500] Loss: 0.6534
Epoch [5/80], Step [200/500] Loss: 0.6046
Epoch [5/80], Step [300/500] Loss: 0.5702
Epoch [5/80], Step [400/500] Loss:

In [12]:
# Test the model
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Accuracy of the model on the test images: {} %'.format(100 * correct / total))

Accuracy of the model on the test images: 92.99 %


In [13]:
model = ResNet(Bottleneck, [3, 4, 6, 3]).to(device)

In [14]:
# Loss and optimizer
num_epochs = 80
learning_rate = 0.001
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [15]:
# For updating learning rate
def update_lr(optimizer, lr):    
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

In [16]:
# Train the model
total_step = len(train_loader)
curr_lr = learning_rate
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i+1) % 100 == 0:
            print ("Epoch [{}/{}], Step [{}/{}] Loss: {:.4f}"
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

    # Decay learning rate
    if (epoch+1) % 20 == 0:
        curr_lr /= 3
        update_lr(optimizer, curr_lr)

Epoch [1/80], Step [100/500] Loss: 1.8722
Epoch [1/80], Step [200/500] Loss: 1.5546
Epoch [1/80], Step [300/500] Loss: 1.4912
Epoch [1/80], Step [400/500] Loss: 1.3069
Epoch [1/80], Step [500/500] Loss: 1.1303
Epoch [2/80], Step [100/500] Loss: 1.1997
Epoch [2/80], Step [200/500] Loss: 1.3460
Epoch [2/80], Step [300/500] Loss: 1.1125
Epoch [2/80], Step [400/500] Loss: 0.9988
Epoch [2/80], Step [500/500] Loss: 0.9444
Epoch [3/80], Step [100/500] Loss: 0.8973
Epoch [3/80], Step [200/500] Loss: 1.0010
Epoch [3/80], Step [300/500] Loss: 1.0233
Epoch [3/80], Step [400/500] Loss: 0.7729
Epoch [3/80], Step [500/500] Loss: 0.8495
Epoch [4/80], Step [100/500] Loss: 0.8925
Epoch [4/80], Step [200/500] Loss: 0.8603
Epoch [4/80], Step [300/500] Loss: 0.6710
Epoch [4/80], Step [400/500] Loss: 0.6513
Epoch [4/80], Step [500/500] Loss: 0.5553
Epoch [5/80], Step [100/500] Loss: 0.6803
Epoch [5/80], Step [200/500] Loss: 0.7425
Epoch [5/80], Step [300/500] Loss: 0.6266
Epoch [5/80], Step [400/500] Loss:

In [17]:
# Test the model
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Accuracy of the model on the test images: {} %'.format(100 * correct / total))

Accuracy of the model on the test images: 92.76 %
